# Teardown ComfyUI deployment on RunPod (A5000)

This notebook contains a safe teardown flow for a deployment created with the new A5000 deployment script. By default it is a dry run.

**Assumptions:**

- `runpodctl` is installed and you are authenticated.

- The deployment name below matches the one you created.

In [ ]:
import subprocess, os

DRY_RUN = True  # Set to False to actually run the delete command

def run(cmd):
    print('\n$ ' + ' '.join(cmd))
    
    if not DRY_RUN:
        subprocess.run(cmd, check=True)

## Delete the deployment

Change `deployment_name` to match your instance. The `deployments delete` subcommand is used as a reasonable assumption.

In [ ]:
deployment_name = 'comfyui-staging-a5000'

run(['runpodctl', 'deployments', 'delete', '--name', deployment_name])

if DRY_RUN:
    print('\nDry run complete — set DRY_RUN = False to execute')

## Alternative: Terminate individual pods

If you created individual pods instead of deployments, you can terminate them using the Python API:

In [ ]:
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('.'))))
sys.path.insert(0, project_root)

from src.infra.runpod_manager import RunPodManager

def terminate_comfy_pods():
    """Terminate all ComfyUI pods."""
    try:
        # Create an instance of RunPodManager
        manager = RunPodManager()
        print("✅ RunPod manager initialized")
        
        # List all active pods
        print("\n📋 Getting active pods...")
        active_pods = manager.get_active_pods()
        
        # Filter for ComfyUI pods
        comfy_pods = [pod for pod in active_pods if 'comfyui' in pod.get('name', '').lower()]
        
        if not comfy_pods:
            print("✅ No active ComfyUI pods found.")
            return
        
        print(f"Found {len(comfy_pods)} active ComfyUI pod(s):")
        print("-" * 30)
        
        # Display pod information
        for i, pod in enumerate(comfy_pods, 1):
            print(f"{i}. Name: {pod.get('name', 'N/A')}")
            print(f"   ID: {pod.get('id', 'N/A')}")
            print(f"   Status: {pod.get('desiredStatus', 'N/A')}")
            print(f"   GPU: {pod.get('gpuTypeId', 'N/A')}")
            print(f"   Interruptible: {pod.get('interruptible', False)}")
            print()
        
        # Terminate all ComfyUI pods
        print("🛑 Terminating ComfyUI pods...")
        for pod in comfy_pods:
            pod_id = pod.get('id')
            pod_name = pod.get('name', 'Unknown')
            print(f"Terminating pod: {pod_name} (ID: {pod_id})...")
            try:
                manager.terminate_pod(pod_id)
                print(f"  ✅ Pod {pod_id} termination requested")
            except Exception as e:
                print(f"  ❌ Error terminating pod {pod_id}: {e}")
                
    except Exception as e:
        print(f"❌ Error: {e}")

# Uncomment the line below to run termination
# terminate_comfy_pods()